In [2]:
!pip install mediapipe 
!pip install opencv-python

In [8]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

#to get real time feed from our camera
cap = cv2.VideoCapture(0) #video capture and the parameter is the number which my webcam is represented
while cap.isOpened():
    ret, frame = cap.read() 
    cv2.imshow('Mideapipe Feed', frame)
    if cv2.waitKey(10) & 0xFF == ord('x'):
        break

cap.release()
cv2.destroyAllWindows()

In [9]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose




In [10]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

NameError: name 'landmarks' is not defined

In [11]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

NameError: name 'landmarks' is not defined

In [12]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

NameError: name 'landmarks' is not defined

In [13]:

#calculating the angle for left shoulder elbow and wrist
def calculate_angle(a,b,c):
    a = np.array(a) #first
    b = np.array(b) #mid
    c = np.array(c) #end

    radians = np.arctan2(c[1] - b[1], c[0] - b[0])- np.arctan2(a[1]-b[1], a[0] - b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360 - angle
    return angle            

In [14]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            #angle calculation
calculate_angle(shoulder, elbow, wrist)

NameError: name 'landmarks' is not defined

In [30]:
#applying mediapipe component to make detetctions

cap = cv2.VideoCapture(0) 

#bicep curl count
reps = 0
stage = None

with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read() 
        if not ret:
            break
    #detection and render
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable  = False
#make detection and stored in variable results
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
#landmark extrcation
        try:
            landmarks = results.pose_landmarks.landmark
            #now passing (shoulder, elbow, wrist) --> (a,b,c) respectively as co-ordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            #angle calculation
            angle = calculate_angle(shoulder,elbow,wrist)
            #visualisation (passing through the image, angle which is converted to a string, and the positioning the angle and putting in a tuple because cv2 works on tuples
            cv2.putText(image,str(angle),
                        tuple(np.multiply(elbow,[640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (30,245,255), 2, cv2.LINE_AA
                              )    
                      #angle seems to be fine but elbow not tracking, fix it!
            #counting each bicep curl repetition
            if angle > 160:
                stage = "Down"
            if angle< 30 and stage == "Down":
                stage = "Up"
                reps += 1
                print(reps)
            
        except:
            pass

        #adding the repetition count in the frame
        
        cv2.putText(image, "REPS", (10,90),#added the co-ordinates
                    cv2.FONT_ITALIC, 0.5,  (0,0,0), 1, cv2.LINE_AA)#added the font, size of text, text color, line width, line type
        cv2.putText(image, str(reps), (10,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        #arm positioning
        cv2.putText(image, "POSITION", (80,90),#added the co-ordinates
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5,  (0,0,0), 1, cv2.LINE_AA)#added the font, size of text, text color, line width, line type
        cv2.putText(image, stage, (60,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        #detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color = (245,117,66), thickness = 2, circle_radius = 2), #joints
                                 mp_drawing.DrawingSpec(color = (245,66,230), thickness = 2, circle_radius = 2) #connections
                                 )
        
        
        
        cv2.imshow('Mideapipe Feed', image)
        if cv2.waitKey(1) & 0xFF == ord('x'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1727308492.603086 5546086 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2
W0000 00:00:1727308492.692612 5558261 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1727308492.706927 5558261 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


1
2
